In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np

from analyze_decode_swrs import get_decoded_swr_plots

In [ ]:
import info.r063d6 as r063d6
import info.r063d7 as r063d7
infos = [r063d6, r063d7]

In [ ]:
from run import (analysis_infos,
                 r063_infos, r066_infos, r067_infos, r068_infos,
                 days1234_infos, days5678_infos,
                 day1_infos, day2_infos, day3_infos, day4_infos, day5_infos, day6_infos, day7_infos, day8_infos)

# import info.r066d2 as r066d2
# import info.r068d8 as r068d8
# infos = [r068d8, r066d2]
# group = "test"
# get_decoded_swr_plots(infos, group=group, update_cache=True)

info_groups = dict()
# info_groups["All"] = analysis_infos
info_groups["R063"] = r063_infos
info_groups["R066"] = r066_infos
info_groups["R067"] = r067_infos
info_groups["R068"] = r068_infos
# info_groups["Days1234"] = days1234_infos
# info_groups["Days5678"] = days5678_infos
# info_groups["Day1"] = day1_infos
# info_groups["Day2"] = day2_infos
# info_groups["Day3"] = day3_infos
# info_groups["Day4"] = day4_infos
# info_groups["Day5"] = day5_infos
# info_groups["Day6"] = day6_infos
# info_groups["Day7"] = day7_infos
# info_groups["Day8"] = day8_infos

# get_decoded_swr_plots(analysis_infos, group="All", z_thresh=1., power_thresh=2., update_cache=True)


get_decoded_swr_plots(infos, group="All", z_thresh=2., power_thresh=3., update_cache=True)

In [ ]:
u = []
len(u)

In [ ]:
for infos, group in zip(info_groups.values(), info_groups.keys()):
    get_decoded_swr_plots(infos, group, z_thresh=1., power_thresh=2., update_cache=False)

for power_thresh in [3, 4, 5]:
    get_decoded_swr_plots(analysis_infos, group="All", power_thresh=power_thresh, update_cache=True)

    for infos, group in zip(info_groups.values(), info_groups.keys()):
        get_decoded_swr_plots(infos, group, power_thresh=power_thresh, update_cache=False)

In [ ]:
10+10+15+5